In [1]:
from pandas import *
from os import listdir, getcwd
from os.path import join
from json import loads
from nltk import *
from nltk import download
from datetime import date, datetime
import pickle
download('stopwords')
download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
transcript_dir = 'resources/hackathon_data/company_transcripts/'
price_dir = 'resources/hackathon_data/company_prices_returns/'
WORD_THRESHOLD = 20

In [3]:
transcript_files = listdir(transcript_dir)
price_files = listdir(price_dir)

In [4]:
def cut_word_threshold(lst, threshold):
    return [x for x in lst if len(x.split()) >= threshold]

In [5]:
df_all = DataFrame(columns=['ticker', 'datetime', 'date', 'year', 'quarter', 'close', 'returns', 'next_returns', 'usable_returns', 'body', 'earnings_word_count'])

In [6]:
# Load all data
for index in range(len(transcript_files)):
    trans_file = transcript_files[index]
    price_file = price_files[index]
    
    # Get ticker
    ticker = trans_file.split('.')[0]

    # Grab data
    df_trans_raw = read_json(transcript_dir + trans_file)[['date', 'body']]
    df_price_raw = read_csv(price_dir + price_file)
    
    if len(df_trans_raw) == 0 or len(df_price_raw) == 0:
        continue
    
    df_trans = df_trans_raw
    df_price = df_price_raw
    # Rename Columns
    df_trans.columns = ['datetime', 'body']
    df_price.columns = ['date', 'close', 'returns']

    # Convert to datetime.date
    df_trans['date'] = df_trans['datetime'].apply(lambda x: x.date())
    # Convert to datetime.date
    df_price['date'] = df_price['date'].apply(lambda x: date(*[int(y) for y in x.split('-')]))
    # Shift returns from next day to current
    df_price['next_returns'] = df_price['returns'].shift(-1)
    
    # Merge pricing with earnings data
    df_merged = df_trans.merge(df_price, left_on=['date'], right_on=['date'], how='left')

    # Get returns we want to use
    df_merged['usable_returns'] = df_merged.apply(lambda x: x['next_returns'] if x['datetime'].hour >= 16 else x['returns'], axis=1)
    
    df_merged['ticker'] = ticker
    
    df_merged['year'] = df_merged['date'].apply(lambda x: x.year)
    df_merged['quarter'] = df_merged['date'].apply(lambda x: 1 if x.month <= 3 else 2 if x.month <= 6 else 3 if x.month <= 9 else 4)
    
    df_merged['body'] = df_merged['body'].apply(lambda lst: ' '.join(cut_word_threshold(lst, WORD_THRESHOLD)))
    
    df_merged['earnings_word_count'] = df_merged['body'].apply(lambda x: len(x.split()))
    
    df_all = df_all.append(df_merged, sort=False)


In [7]:
df_all[['ticker', 'datetime', 'date', 'year', 'quarter', 'close', 'returns', 'next_returns', 'usable_returns', 'body']]

,ticker,datetime,date,year,quarter,close,returns,next_returns,usable_returns,body
0,A,2012-02-15 16:30:00,2012-02-15,2012,1,8.890000,-0.012222,0.023622,0.023622,"Good day, ladies and gentlemen, and welcome to..."
1,A,2012-08-15 16:30:00,2012-08-15,2012,3,10.150000,0.002964,-0.023645,-0.023645,"Good day, ladies and gentlemen, and welcome to..."
2,A,2012-11-19 16:30:00,2012-11-19,2012,4,12.070000,0.026360,0.034797,0.034797,"Good day, ladies and gentlemen, and welcome to..."
3,A,2013-02-14 16:30:00,2013-02-14,2013,1,13.990000,-0.045703,0.036455,0.036455,"Good day, ladies and gentlemen, and welcome to..."
4,A,2013-05-14 16:30:00,2013-05-14,2013,2,18.100000,0.021445,0.039226,0.039226,"Good day, ladies and gentlemen, and welcome to..."
5,A,2013-08-14 16:30:00,2013-08-14,2013,3,16.170000,-0.011614,-0.027829,-0.027829,"Good day, ladies and gentlemen, and welcome to..."
6,A,2015-11-17 16:30:00,2015-11-17,2015,4,42.299999,-0.012375,-0.013002,-0.013002,At this time all participants are in a listen-...
7,A,2016-02-16 16:30:00,2016-02-16,2016,1,38.610001,0.020888,0.018907,0.018907,"Good day, ladies and gentlemen, and welcome to..."
8,A,2016-05-16 16:30:00,2016-05-16,2016,2,32.119999,0.019359,0.016189,0.016189,"Good day, ladies and gentlemen, and welcome to..."
9,A,2016-08-17 16:30:00,2016-08-17,2016,3,36.599998,-0.004082,0.001913,0.001913,"Good day, ladies and gentlemen, and welcome to..."


In [8]:
df_all.to_csv('all_merged_data.csv')